Context: We are Data Scientist at "MusicStream."\
Goal: Prove the value of a recommendation algorithm using rigorous statistical methods.\
Tools: Python (Pandas, Plotly, SciPy, Lifelines).

In [1]:
pip install faker plotly lifelines scipy numpy pandas

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 4.4 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=9c5dc1457e1f6a38bf69ceaacee18e58f692d37e0adf4be771139fd227eba645
  Stored in directory: /root/.cache/pip/wheels/50/37/21/0a719b9d89c635e89ff24bd93b862882ad675279552013b2fb
Successfully built autograd-gamma


In [20]:
pip install plotly statsmodels

In [46]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
from statsmodels.stats.power import TTestIndPower
from lifelines import KaplanMeierFitter
from faker import Faker
from statsmodels.stats.proportion import proportions_ztest

# Configuration for reproducibility
np.random.seed(42)
fake = Faker()

***Phase 1 - Power Analysis***

We calculate how many users we need. This prevents "Underpowered" tests.

In [47]:
# Experiment Parameters
effect_size = 0.05   # Small effect (Cohen's d)
alpha = 0.05         # 5% significance level
power = 0.80         # 80% power

# Unbalanced Split (90% Control / 10% Treatment)
# Ratio = Treatment Size / Control Size
treatment_ratio = 0.10 / 0.90

analysis = TTestIndPower()
# Solve for Control Group Size
n_control = analysis.solve_power(effect_size=effect_size, power=power, alpha=alpha, ratio=treatment_ratio)
n_treatment = n_control * treatment_ratio

total_required = int(n_control + n_treatment)

print(f"--- Experiment Design (90/10 Ramp-Up) ---")
print(f"Control Users Needed:   {int(n_control):,}")
print(f"Treatment Users Needed: {int(n_treatment):,}")
print(f"Total Users Needed:     {total_required:,}")
print("\nInsight: Unbalanced tests require MORE total users than 50/50 tests to achieve the same statistical power.")

--- Experiment Design (90/10 Ramp-Up) ---
Control Users Needed:   31,397
Treatment Users Needed: 3,488
Total Users Needed:     34,885

Insight: Unbalanced tests require MORE total users than 50/50 tests to achieve the same statistical power.


***Phase 1 - A/A Test Simulation (The Baseline Check)***

Before running the real test, we simulate 1,000 "null" tests to prove our stats engine doesn't generate false positives.

In [48]:
p_values = []
# Run 1000 simulations
for _ in range(1000):
    # Generate two identical groups (No treatment effect)
    group_a = np.random.normal(20, 5, 1000)
    group_b = np.random.normal(20, 5, 1000)

    # Run T-Test
    _, p = stats.ttest_ind(group_a, group_b)
    p_values.append(p)

# Calculate False Positive Rate (Target: ~5%)
fp_rate = sum(x < 0.05 for x in p_values) / 1000

print(f"False Positive Rate: {fp_rate:.1%} (Target: 5.0%)")

if 0.04 <= fp_rate <= 0.06:
    print("System Calibrated: We are ready for the A/B test.")
else:
    print("Calibration Warning: Review randomization logic.")

False Positive Rate: 4.7% (Target: 5.0%)
System Calibrated: We are ready for the A/B test.


***Phase 2 - Data Engineering***

This generates the Raw Data (Users + Events) AND aggregates into a table for analysis

In [49]:
def generate_industry_data(n_users=40000):
    print(f"Generating raw logs for {n_users:,} users...")

    # --- 1. USERS TABLE ---
    groups = np.random.choice(['Control', 'Treatment'], size=n_users, p=[0.90, 0.10])

    df_users = pd.DataFrame({
        'user_id': range(n_users),
        'group': groups,
        'country': np.random.choice(['US', 'UK', 'DE', 'IN'], n_users, p=[0.4, 0.2, 0.2, 0.2]),
        'device': np.random.choice(['iOS', 'Android'], n_users, p=[0.5, 0.5]),
        'age_group': np.random.choice(['18-24', '25-34', '35-44', '45+'], n_users, p=[0.35, 0.35, 0.2, 0.1]) #skewed data as music customers are more young
    })

    # --- 2. EVENTS LOG ---
    activity_counts = np.zeros(n_users, dtype=int)
    mask_c = df_users['group'] == 'Control'
    mask_t = df_users['group'] == 'Treatment'

    # Treatment Effect: 15% Lift in Activity
    activity_counts[mask_c] = np.random.gamma(2, 10, size=mask_c.sum()).astype(int)
    activity_counts[mask_t] = np.random.gamma(2, 11.5, size=mask_t.sum()).astype(int)

    # ETL (Aggregation)
    df_users['songs_played'] = activity_counts

    # --- 3. CONVERSION DATA ---
    df_users['propensity'] = 0.05 + (df_users['songs_played'] * 0.002) ## base: conversion is based on songs played
    df_users.loc[df_users['country'] == 'US', 'propensity'] *= 1.2 ## bias: US people tend to spend more
    df_users['converted'] = np.random.binomial(1, df_users['propensity'].clip(upper=1.0))

    # --- 4. RETENTION DATA ---
    # Logic:
    # - Everyone starts with random tenure (1-60 days)
    # - 'Converted' users stay longer (Lower churn)
    # - 'Treatment' users stay slightly longer (Better experience)

    # Base tenure
    df_users['days_active'] = np.random.randint(1, 60, size=n_users)

    # Churn Probability Calculation
    # Base churn risk = 30%
    churn_risk = np.full(n_users, 0.30)

    # Premium users are much less likely to churn (Risk -20%)
    churn_risk[df_users['converted'] == 1] -= 0.20

    # Treatment group is slightly less likely to churn (Risk -5%)
    churn_risk[df_users['group'] == 'Treatment'] -= 0.05

    # Simulate Churn Event (1 = Churned/Left, 0 = Still Active/Censored)
    df_users['churned'] = np.random.binomial(1, churn_risk.clip(0, 1))

    # --- 5. INJECT OUTLIERS ---
    outlier_indices = np.random.choice(df_users.index, size=50, replace=False)
    df_users.loc[outlier_indices, 'songs_played'] = np.random.randint(1000, 5000, size=50)

    return df_users

# Generate the Master Dataset
df_raw = generate_industry_data()
display(df_raw.head())

Generating raw logs for 40,000 users...


,user_id,group,country,device,age_group,songs_played,propensity,converted,days_active,churned
0,0,Control,DE,Android,35-44,17,0.0840,0,11,0
1,1,Control,US,Android,45+,6,0.0744,0,15,0
2,2,Control,IN,iOS,18-24,20,0.0900,1,51,1
3,3,Control,US,iOS,25-34,14,0.0936,0,30,0
4,4,Control,US,iOS,25-34,6,0.0744,0,43,0


***Phase 2 - Outlier Detection & Cleaning***

Visualizing and removing outliers.

In [50]:
# 1. Visualize Boxplot
fig = px.box(df_raw, x='group', y='songs_played',
             title='Boxplot of Songs Played (Before Cleaning)',
             template='plotly_white')
fig.show()

# 2. Define Threshold (99th Percentile)
threshold = df_raw['songs_played'].quantile(0.99)
print(f"99th Percentile Threshold: {threshold:.0f} songs")

# 3. Filter Data
df_clean = df_raw[df_raw['songs_played'] < threshold].copy()
removed = len(df_raw) - len(df_clean)

print(f"Removed {removed} outliers (Bots/Whales).")
print(f"Cleaned Dataset Size: {len(df_clean):,}")

99th Percentile Threshold: 69 songs
Removed 419 outliers (Bots/Whales).
Cleaned Dataset Size: 39,581


***Why we need to remove outliers?***



***Phase 3 - Guardrails (SRM Check)***

Validating the 90/10 split. If this fails, the experiment is broken.

In [51]:
# Observed Counts
obs = df_clean['group'].value_counts()
# Order: [Control, Treatment] to match expectation logic below
obs_values = [obs['Control'], obs['Treatment']]

# Expected Counts (90% Control, 10% Treatment)
total_clean = len(df_clean)
exp_values = [total_clean * 0.90, total_clean * 0.10]

print(f"Observed: {obs_values}")
print(f"Expected: {exp_values}")

chi2, p_val = stats.chisquare(f_obs=obs_values, f_exp=exp_values)
print(f"P-Value: {p_val:.5f}")

if p_val < 0.01:
    print("SRM WARNING: The split is not 90/10!")
else:
    print("SRM PASSED: Groups are balanced correctly.")

Observed: [np.int64(35637), np.int64(3944)]
Expected: [35622.9, 3958.1000000000004]
P-Value: 0.81325
SRM PASSED: Groups are balanced correctly.


***Why we do chisquare test?***\
It is a "Sanity Check" to ensure the experiment isn't broken.

1. The Logic: Expected vs. Observed

When we design an A/B test, we program it to split traffic 10/90.

If we have 10,000 users, we expect 9,000 in Control and 1,000 in Treatment.

However, in the real world, bugs happen. Maybe the new feature crashes on older Android phones, so those users never get logged into the "Treatment" group.

Is the user difference just random luck? Or is the randomization engine broken? The Chi-Square Test answers this question. It compares the numbers we Observed against the numbers we Expected.

2. How to Read the Result

High P-Value (> 0.01): The difference is small. It's likely just random noise. The experiment is VALID.

Low P-Value (< 0.01): The difference is huge. It is extremely unlikely to happen by chance. The experiment is BROKEN.

Why this matters: If our sample is mismatched (e.g., 55% Control / 45% Treatment), it often means a specific type of user (e.g., Chrome users, iPhone users) is failing to enter the experiment. If we analyze the data anyway, our results will be wrong.

***Phase 4 - Statistical Inference (T-Test)***

Analyzing the Primary Metric (Songs Played). Note the use of equal_var = False

In [52]:
control = df_clean[df_clean['group']=='Control']['songs_played']
treat = df_clean[df_clean['group']=='Treatment']['songs_played']

# Welch's T-Test (equal_var=False)
t_stat, p_val = stats.ttest_ind(control, treat, equal_var=False)
lift = (treat.mean() - control.mean()) / control.mean()

print(f"Lift: {lift:.2%}")
print(f"P-Value: {p_val:.5f}")

if p_val < 0.05:
    print("RESULT: Statistically Significant Increase.")

Lift: 13.26%
P-Value: 0.00000
RESULT: Statistically Significant Increase.


***Why Welch's T-Test?***

1. Why a t-test?

In a real experiment, we do not know the true population standard deviation. We only know the sample standard deviation calculated from data. Therefore, the t-test is the theoretically correct test.

2. But isn't N large?

Yes. With 40,000 users, the t-distribution becomes almost identical to the Z-distribution. However, running a t-test is safer and never "wrong," whereas running a Z-test when we don't actually know is technically an approximation (though a very good one at this sample size).

3. Why equal_var=False?

This enables Welch's t-test, which does not assume the two groups have the same variance. This is critical because our sample sizes are unequal (90% vs 10%).

***Phase 4 - Conversion Analysis (Z-Test)***

Analyzing the Secondary Metric (Purchase Rate)

In [53]:
convs = df_clean.groupby('group')['converted'].sum()
nobs = df_clean.groupby('group')['converted'].count()

# Ensure order [Treatment, Control]
count = [convs['Treatment'], convs['Control']]
n_obs = [nobs['Treatment'], nobs['Control']]

stat, p_z = proportions_ztest(count, n_obs)

print(f"Treatment Conv Rate: {count[0]/n_obs[0]:.2%}")
print(f"Control Conv Rate:   {count[1]/n_obs[1]:.2%}")
print(f"P-Value: {p_z:.5f}")

Treatment Conv Rate: 10.29%
Control Conv Rate:   9.68%
P-Value: 0.21358


***Why Z-Test for Conversion?***

For binary data (converted vs. not converted), the variance is mathematically tied to the mean ($p(1-p)$). Because we can calculate the variance directly from the sample proportion, and sample sizes are large, the Central Limit Theorem allows us to use the Z-test.

***Result Interpretation***

We cannot prove that the new feature increases conversion. While the data looks directionally positive, the effect is too weak to be distinguished from random noise. We should NOT expect a guaranteed revenue increase if we launch.

***What if p-value is close to 0.05 but greater than 0.05?***

1. Explanation: What does a P-Value of 0.06 mean?
P-value of 0.06 is a very tricky result. It is often called "Directionally Positive but Not Statistically Significant."

The Math: It means there is a 6% chance that the improvement you see is just random luck.

The Standard: Most companies set the "Significance Level" (Alpha) at 0.05 (5%). Because 0.06 > 0.05, you technically failed the test. You cannot claim the feature definitely increases revenue.

Business Interpretation:

"We are 94% sure this feature helps, but our safety standard requires us to be 95% sure. If we launch this, there is a moderate risk (6%) that the revenue boost isn't real."

What would we do in a real job?

Do not launch yet.

Run the test longer: We might just need more sample size to push that 0.06 down to 0.04.

Check Segmentation: Maybe it is significant (p < 0.05) for iOS users but not Android. Launch only for iOS.

***Phase 5 - Deep Dive (Segmentation)***

Checking for Heterogeneous Effects

e.g. Does it fail on Android?

In [55]:
# Calculate Lift by Device
seg = df_clean.groupby(['device', 'group'])['songs_played'].mean().unstack()
seg['Lift'] = (seg['Treatment'] - seg['Control']) / seg['Control']

display(seg)

# Plot
fig = px.bar(seg.reset_index(), x='device', y='Lift',
             title='Treatment Lift by Device (Segmentation)',
             text_auto='.1%', color='Lift', template='plotly_white')
fig.show()

group,Control,Treatment,Lift
device,,,
Android,19.073688,21.519026,0.128205
iOS,19.025498,21.632032,0.137002


Investigating if the feature works differently for different Age Groups.

In [56]:
# Group by Age and Experiment Group
segmentation = df_clean.groupby(['age_group', 'group'])['songs_played'].mean().unstack()

# Calculate Lift per Segment
segmentation['Lift'] = (segmentation['Treatment'] - segmentation['Control']) / segmentation['Control']
segmentation['Lift %'] = segmentation['Lift'].apply(lambda x: f"{x*100:.2f}%")

display(segmentation[['Control', 'Treatment', 'Lift %']])

# Visualizing the Lift by Age
fig = px.bar(segmentation.reset_index(), x='age_group', y='Lift',
             title='Impact of New Algorithm by Age Group',
             labels={'Lift': 'Relative Lift in Songs Played'},
             color='Lift', color_continuous_scale='RdBu',
             template='plotly_white')
fig.show()

group,Control,Treatment,Lift %
age_group,,,
18-24,19.197078,21.482659,11.91%
25-34,18.999285,21.719708,14.32%
35-44,18.866676,21.437186,13.62%
45+,19.079895,21.680203,13.63%


***Phase 6 - Visualization (Sankey Diagram)***

Visualizing the User Flow for the Treatment Group.

In [54]:
# 1. Filter Data for Treatment Group Only
df_treat = df_clean[df_clean['group'] == 'Treatment']

# 2. Calculate Funnel Counts
total_users = len(df_treat)
# Users who did ANY activity (Songs > 0)
active_users = len(df_treat[df_treat['songs_played'] > 0])
# Users who paid
premium_users = df_treat['converted'].sum()

# Derived Metrics (Drop-offs)
bounced_users = total_users - active_users       # Users with 0 songs
free_users = active_users - premium_users        # Active but didn't pay

# 3. Define Nodes and Colors
# Indices: 0=Start, 1=Active, 2=Premium, 3=Bounce, 4=Free
labels = ["App Open", "Active (Played Songs)", "Premium Purchase", "Bounced (0 Songs)", "Remained Free"]
colors = ["blue", "blue", "green", "red", "gray"]

# 4. Define Links (Source -> Target)
source = [0, 0, 1, 1]
target = [1, 3, 2, 4]
values = [
    active_users,    # Open -> Active
    bounced_users,   # Open -> Bounce
    premium_users,   # Active -> Premium
    free_users       # Active -> Free
]

# 5. Plot
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 20, thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = labels,
      color = colors
    ),
    link = dict(
      source = source,
      target = target,
      value = values,
      color = ["rgba(0,0,255,0.3)", "rgba(255,0,0,0.3)", "rgba(0,128,0,0.5)", "rgba(128,128,128,0.3)"]
    ))])

fig.update_layout(title_text=f"User Conversion Flow (Treatment Group, N={total_users})", font_size=12)
fig.show()

**Survival Analysis (Retention)**\
Visualizing how long users stick around using Kaplan-Meier curves.

In [57]:
kmf = KaplanMeierFitter()
fig = go.Figure()

# 1. Fit Control Group
# We filter our cleaned data for Control users
data_control = df_clean[df_clean['group'] == 'Control']

kmf.fit(
    durations=data_control['days_active'],
    event_observed=data_control['churned'],
    label='Control'
)

# Get the probability data
surv_control = kmf.survival_function_
fig.add_trace(go.Scatter(
    x=surv_control.index, y=surv_control['Control'],
    mode='lines', name='Control Group', line=dict(color='blue')
))

# 2. Fit Treatment Group
data_treat = df_clean[df_clean['group'] == 'Treatment']

kmf.fit(
    durations=data_treat['days_active'],
    event_observed=data_treat['churned'],
    label='Treatment'
)

surv_treat = kmf.survival_function_
fig.add_trace(go.Scatter(
    x=surv_treat.index, y=surv_treat['Treatment'],
    mode='lines', name='Treatment Group', line=dict(color='orange', dash='dash')
))

# 3. Formatting
fig.update_layout(
    title="Kaplan-Meier Survival Curve (User Retention)",
    xaxis_title="Days Since Signup",
    yaxis_title="Probability of Remaining Active",
    template="plotly_white",
    hovermode="x unified"
)

fig.show()

print("Interpretation: If the Orange line is higher than the Blue line, Treatment users stay longer.")

Interpretation: If the Orange line is higher than the Blue line, Treatment users stay longer.


**What's the summary of the current stage?**

We face the engagement - monetization gap.

When Engagement is up but Revenue is flat, usually one of three things is happening:

1. The "Free Ride" Problem: The new feature is too good for free users. They are so happy with the free product that they feel no need to upgrade. We cannibalized your own upsell.

2. The "Power Issue" (False Negative): Revenue events are rare (only 5-10% convert). We might simply lack the sample size to detect a small +2% revenue lift.

3. The "Wrong Segment" Problem: The feature works great for teenagers (who have no money) but fails for adults (who do).

**Next Steps**

**Analysis 1: Segmentation**

Hypothesis: Maybe the revenue lift is significant on a specific segment. If so, the global average washes out the win.

Action: Deep dive and see where is the money


In [59]:
# Define segments to check
segments = ['device', 'country', 'age_group']

for segment in segments:
    print(f"\nAnalyzing by {segment.upper()}:")

    # Get unique values (e.g., 'iOS', 'Android')
    groups = df_clean[segment].unique()

    for g in groups:
        # Filter for just this sub-group
        subset = df_clean[df_clean[segment] == g]

        # Run Z-Test on this subset
        convs = subset.groupby('group')['converted'].sum()
        nobs = subset.groupby('group')['converted'].count()

        # Safety check: Need at least 10 conversions to run stats
        if convs.min() < 10:
            print(f"  - {g}: Not enough data.")
            continue

        stat, p = proportions_ztest([convs['Treatment'], convs['Control']],
                                    [nobs['Treatment'], nobs['Control']])

        lift = (convs['Treatment']/nobs['Treatment']) / (convs['Control']/nobs['Control']) - 1

        # Highlight significant findings
        icon = "✅" if p < 0.05 else "❌"
        print(f"  {icon} {g:<10} | Lift: {lift:+.2%} | P-Value: {p:.4f}")


Analyzing by DEVICE:
  ❌ Android    | Lift: +3.91% | P-Value: 0.5872
  ❌ iOS        | Lift: +8.95% | P-Value: 0.2225

Analyzing by COUNTRY:
  ❌ DE         | Lift: +0.90% | P-Value: 0.9387
  ❌ US         | Lift: +10.25% | P-Value: 0.1810
  ❌ IN         | Lift: -6.97% | P-Value: 0.5608
  ❌ UK         | Lift: +16.34% | P-Value: 0.1873

Analyzing by AGE_GROUP:
  ❌ 35-44      | Lift: -1.76% | P-Value: 0.8751
  ❌ 45+        | Lift: +13.05% | P-Value: 0.4192
  ❌ 18-24      | Lift: +6.56% | P-Value: 0.4531
  ❌ 25-34      | Lift: +9.15% | P-Value: 0.2977


**Analysis 2: Proxy Metric Analysis**

Hypothesis: Revenue is a "Lagging Indicator" (it takes 30 days to decide to pay). Engagement is a "Leading Indicator."

Action: Check "Add to Playlist" or "Share Song" rates. If these high-intent actions are up, revenue will follow eventually (LTV).

In [60]:
# 1. Define "Power Users" (Top 25% of listeners)
threshold = df_clean['songs_played'].quantile(0.75)
power_users = df_clean[df_clean['songs_played'] >= threshold]

# 2. Check Conversion Lift specifically for Power Users
convs = power_users.groupby('group')['converted'].sum()
nobs = power_users.groupby('group')['converted'].count()

stat, p = proportions_ztest([convs['Treatment'], convs['Control']],
                            [nobs['Treatment'], nobs['Control']])

rate_t = convs['Treatment'] / nobs['Treatment']
rate_c = convs['Control'] / nobs['Control']
lift = (rate_t - rate_c) / rate_c

print(f"Power User Conv Rate (Control):   {rate_c:.2%}")
print(f"Power User Conv Rate (Treatment): {rate_t:.2%}")
print(f"Lift: {lift:+.2%}")
print(f"P-Value: {p:.4f}")

if p < 0.05:
    print("SIGNAL FOUND: The feature works for Power Users!")
    print("Recommendation: Launch. These users drive long-term value.")
else:
    print("Still not significant.")

Power User Conv Rate (Control):   13.73%
Power User Conv Rate (Treatment): 14.62%
Lift: +6.49%
P-Value: 0.3968
Still not significant.


**Analysis 3: Cannibalization Check**

Hypothesis: Did the treatment group view fewer "Upgrade Now" paywalls because the algorithm kept them listening smoothly?

Action: Check the number of paywall impressions.

While this didn't drive immediate upgrades, the 13% lift in listening time increases our Retention. Higher retention historically leads to higher Lifetime Value (LTV). We accept the flat revenue now to gain loyalty. (depends on the feature launching goal & product goal)

In the real world, data is rarely black and white. If Engagement is GREEN and Revenue is GRAY, we usually launch—because keeping users happy (Engagement) is the best way to make money in the long run. (depends on product manager and leadership business decision)